In [1]:
# Imports and Setup
import os
import sys
import numpy as np
import stumpy
from pathlib import Path

sys.path.insert(0, str(Path("..") / "src"))

from utils import config
from utils.io import load_cleaned, load_method_ready, results_dir, append_csv_row, save_json
from utils.evaluation import compute_binary_metrics
from utils.plotting import plot_signal, plot_scores

In [2]:
# Helper: Matrix Profile utilities and Window Mapping 

def mp_to_dist(mp) -> np.ndarray:
    """
    Extract distance column from STUMPY output and convert non-finite to NaN.
    Works for typical (n_subseq, k*2 + 2) MP arrays where dist is column 0.
    """
    dist = np.asarray(mp)[:, 0].astype(float)
    dist[~np.isfinite(dist)] = np.nan
    return dist


def scores_at_starts(dist_all: np.ndarray, starts: np.ndarray) -> np.ndarray:
    """
    dist_all: MP distance for all subsequence positions (0..n-m)
    starts: subsequence start indices to sample
    Returns scores aligned to `starts` length, with NaN for invalid starts.
    """
    starts = np.asarray(starts).astype(int).reshape(-1)
    out = np.full(len(starts), np.nan, dtype=float)
    ok = (starts >= 0) & (starts < len(dist_all))
    out[ok] = dist_all[starts[ok]]
    return out


def windows_scores_to_point_scores_max(
    win_starts_local: np.ndarray,
    win_size: int,
    scores_win: np.ndarray,
    n_points: int,
) -> np.ndarray:
    """
    Point score = max window score covering each point.
    Efficient O(n) using a deque.

    win_starts_local are in test coordinates (0..len(test)-1).
    scores_win aligned with win_starts_local.
    """
    win_starts_local = np.asarray(win_starts_local).astype(int).reshape(-1)
    scores_win = np.asarray(scores_win).astype(float).reshape(-1)

    start_scores = np.full(n_points, np.nan, dtype=float)
    for s, sc in zip(win_starts_local, scores_win):
        s = int(s)
        if 0 <= s < n_points:
            start_scores[s] = float(sc)

    from collections import deque
    dq = deque()  # indices with decreasing start_scores
    out = np.full(n_points, np.nan, dtype=float)

    # We want max over start_scores[j] for j in [i-win_size+1, i]
    for i in range(n_points):
        if not np.isnan(start_scores[i]):
            while dq and start_scores[dq[-1]] <= start_scores[i]:
                dq.pop()
            dq.append(i)

        left = i - win_size + 1
        while dq and dq[0] < max(0, left):
            dq.popleft()

        out[i] = start_scores[dq[0]] if dq else np.nan

    return out


In [3]:
# Main Execution Loop

base_out = results_dir("matrix_profile")
csv_path = base_out / "matrix_profile_results.csv"
if csv_path.exists():
    os.remove(csv_path)

win_size = int(config.MP_PARAMS.get("m", config.WINDOW_SIZE))
q = float(config.BASELINE_THR_QUANTILE)
margin = int(config.PLOT_ZOOM_MARGIN)

for dataset_name in config.DATASETS:
    # Load point labels + split
    _, labels, meta = load_cleaned(dataset_name)
    train_end = int(meta["train_end"])
    y_test = labels[train_end:]

    # Load method-ready arrays
    mr = load_method_ready(dataset_name)
    train_raw = mr["train_raw"]
    test_raw = mr["test_raw"]
    train_robust = mr["train_robust"].astype(float)
    test_robust = mr["test_robust"].astype(float)

    train_win_starts = mr["train_win_starts"].astype(int)  # train coords (0..train_end-1)
    test_win_starts = mr["test_win_starts"].astype(int)    # global coords (0..len(full)-1)
    test_win_labels = mr["test_win_labels"]                # window-level truth in test

    # Convert to test-local window starts
    win_starts_local = test_win_starts - train_end

    out_dir = results_dir("matrix_profile", dataset_name)

    # Overview plot (signal only)
    plot_signal(
        test_raw,
        true_labels=y_test,
        title=f"{dataset_name} - Test (overview)",
        save_path=out_dir / "overview_signal.png",
        max_points=5000,
    )

    # Zoom window (test coords)
    a0 = int(meta["anomaly_start"]) - train_end
    a1 = int(meta["anomaly_end"]) - train_end
    z0 = max(0, a0 - margin)
    z1 = min(len(test_raw), a1 + margin)

    # =========================
    # 1) Train: self-join MP for thresholding
    # =========================
    mp_train = stumpy.stump(train_robust, win_size)
    mp_train_dist_all = mp_to_dist(mp_train)

    train_scores_win = scores_at_starts(mp_train_dist_all, train_win_starts)

    if np.all(~np.isfinite(train_scores_win)):
        raise RuntimeError(f"[{dataset_name}] All train MP scores are non-finite; cannot threshold.")

    thr = float(np.nanquantile(train_scores_win, q))

    # =========================
    # 2) Test: AB-join MP (test vs train) for scoring
    # =========================
    mp_test = stumpy.stump(test_robust, win_size, T_B=train_robust, ignore_trivial=False)
    mp_test_dist_all = mp_to_dist(mp_test)

    test_scores_win = scores_at_starts(mp_test_dist_all, win_starts_local)

    pred_win = (test_scores_win >= thr).astype(int)

    # Window scores -> point scores/preds
    score_point = windows_scores_to_point_scores_max(
        win_starts_local=win_starts_local,
        win_size=win_size,
        scores_win=test_scores_win,
        n_points=len(test_raw),
    )
    pred_point = (np.nan_to_num(score_point, nan=-np.inf) >= thr).astype(int)

    # Metrics
    metrics_point = compute_binary_metrics(y_test, pred_point)
    metrics_win = compute_binary_metrics(test_win_labels, pred_win)

    row = {
        "dataset": dataset_name,
        "method": "matrix_profile",
        "threshold": thr,
        "q": q,
        **metrics_point,
    }
    append_csv_row(csv_path, row)

    save_json(
        out_dir / "matrix_profile_metrics.json",
        {
            **row,
            "window_metrics": metrics_win,
            "mp_window_size": win_size,
            "n_train_windows": int(len(train_win_starts)),
            "n_test_windows": int(len(win_starts_local)),
        },
    )

    # Save artifacts
    np.save(out_dir / "mp_scores_win.npy", test_scores_win)
    np.save(out_dir / "mp_pred_win.npy", pred_win)
    np.save(out_dir / "mp_scores.npy", score_point)
    np.save(out_dir / "mp_pred.npy", pred_point)

    # Plots (zoom)
    plot_signal(
        test_raw[z0:z1], y_test[z0:z1], pred_point[z0:z1],
        title=f"{dataset_name} - Matrix Profile (zoom)",
        save_path=out_dir / "matrix_profile_signal_zoom.png",
        x_offset=z0,
    )
    plot_scores(
        score_point[z0:z1],
        threshold=thr,
        true_labels=y_test[z0:z1],
        title=f"{dataset_name} - MP scores (zoom)",
        save_path=out_dir / "matrix_profile_scores_zoom.png",
        x_offset=z0,
    )